### **Delete (Удаление)**

#### 1. Удаление объектов через ORM

   - **Использование метода `delete()` для удаления записей**:
     Метод `delete()` применяется для удаления одной или нескольких записей в базе данных. Это делается через ORM Django, где запись, выбранная для удаления, указывается через фильтр или напрямую.

   - **Пример удаления отдельного объекта или набора записей по фильтру**:
     Чтобы удалить одну запись, можно получить объект через `get()` и вызвать `delete()` на нем. Для удаления группы объектов применяется `filter()`.

     - **Пример кода**:
       ```python
       # Удаление одного конкретного объекта
       product = Product.objects.get(id=1)
       product.delete()

       # Удаление всех продуктов, у которых цена ниже 10
       Product.objects.filter(price__lt=10).delete()
       ```

   - **Файлы и структура**:
     Поскольку удаление объекта обычно связано с пользователем, его логика включается в представления, а маршрут добавляется в `urls.py`.


#### 2. Удаление через интерфейс

   - **Структура папок и файлы**:
     Для реализации удаления через интерфейс создадим шаблон для подтверждения удаления и добавим кнопку на странице деталей продукта.

     ```bash
     products/
     ├── templates/
     │   └── products/
     │       ├── product_list.html
     │       ├── product_detail.html  # Шаблон с кнопкой удаления
     │       └── product_confirm_delete.html  # Шаблон для подтверждения удаления
     ├── forms.py
     ├── views.py
     ├── urls.py
     ```

   - **Создание представления для подтверждения удаления**:
     В `views.py` добавим представление `product_delete`, которое будет отображать страницу с подтверждением удаления и обрабатывать удаление объекта после подтверждения.

     ```python
     # products/views.py
     from django.shortcuts import render, get_object_or_404, redirect
     from .models import Product

     # Представление для удаления продукта
     def product_delete(request, pk):
         product = get_object_or_404(Product, pk=pk)
         if request.method == 'POST':
             product.delete()  # Удаляем объект
             return redirect('product_list')  # Перенаправление на список продуктов
         return render(request, 'products/product_confirm_delete.html', {'product': product})
     ```

   - **Шаблон для подтверждения удаления**:
     В шаблоне `product_confirm_delete.html` отобразим информацию об объекте и добавим кнопку подтверждения.

     ```html
     <!-- products/templates/products/product_confirm_delete.html -->
     <h1>Удалить {{ product.name }}?</h1>
     <p>Вы уверены, что хотите удалить этот продукт?</p>
     <form method="post">
         {% csrf_token %}
         <button type="submit">Да, удалить</button>
         <a href="{% url 'product_detail' product.id %}">Отмена</a>
     </form>
     ```

   - **Добавление кнопки удаления на странице деталей продукта**:
     На странице `product_detail.html` добавим кнопку для перехода на страницу подтверждения удаления. Кнопка будет перенаправлять пользователя на `product_delete` представление.

     ```html
     <!-- products/templates/products/product_detail.html -->
     <h1>{{ product.name }}</h1>
     <p>Цена: {{ product.price }}</p>
     <p>Описание: {{ product.description }}</p>

     <a href="{% url 'product_edit' product.id %}">Редактировать</a>
     <a href="{% url 'product_delete' product.id %}">Удалить</a>  <!-- Кнопка для удаления -->
     ```

   - **Маршрут для удаления продукта**:
     В `urls.py` добавим URL-маршрут, который будет указывать на `product_delete` представление.

     ```python
     # products/urls.py
     from django.urls import path
     from . import views

     urlpatterns = [
         path('', views.product_list, name='product_list'),
         path('product/<int:pk>/', views.product_detail, name='product_detail'),
         path('product/<int:pk>/edit/', views.product_edit, name='product_edit'),
         path('product/<int:pk>/delete/', views.product_delete, name='product_delete'),  # Маршрут для удаления
     ]
     ```